# Pipeline de 20 etapas em Jupyter para EDA e pré-processamento

Fonte de dados: [Google Drive (pasta)](https://drive.google.com/drive/folders/1Ubbb8K2GuwoHl9TB8VEQgAEzeKMT5nAI)

Este notebook executa 20 etapas, incluindo importação, carregamento, EDA, limpeza, engenharia de atributos, visualizações e exportação do DataFrame processado.


In [ ]:
# 1. Importa81o das bibliotecas necess0rias
import os
import sys
import warnings
from typing import List

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from scipy import stats

warnings.filterwarnings("ignore")

# Ajustes de visualiza81o
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)
sns.set(style="whitegrid", context="notebook")

# Fun85es auxiliares seguras

def cols_exist(df: pd.DataFrame, cols: List[str]) -> List[str]:
    return [c for c in cols if c in df.columns]


def try_parse_datetime(series: pd.Series) -> pd.Series:
    try:
        return pd.to_datetime(series, errors="coerce", dayfirst=True)
    except Exception:
        return pd.to_datetime(series, errors="coerce")


In [ ]:
# 2. Carregamento do arquivo CSV em um DataFrame
from glob import glob

# Defina manualmente se voc souber o caminho do CSV; caso contrrio, o c3digo tentará detectar
BASE_DIR = "/Users/luigigarotti/Desktop/aula compt2"
CSV_PATH = None  # exemplo: "/Users/luigigarotti/Desktop/aula compt2/seu_arquivo.csv"


def detect_csv(base_dir: str) -> str:
    candidates = []
    for pattern in ("*.csv", "**/*.csv"):
        candidates.extend(glob(os.path.join(base_dir, pattern), recursive=True))
    candidates = [p for p in candidates if os.path.isfile(p)]
    if not candidates:
        raise FileNotFoundError(
            "Nenhum arquivo CSV encontrado. Defina CSV_PATH manualmente."
        )
    candidates.sort(key=lambda p: (-os.path.getmtime(p), len(p)))
    return candidates[0]


def read_csv_try_encodings(path: str) -> pd.DataFrame:
    last_err = None
    for enc in ("utf-8", "utf-8-sig", "latin-1", "cp1252"):
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False, engine="python")
        except Exception as e:
            last_err = e
    raise last_err


if CSV_PATH is None:
    CSV_PATH = detect_csv(BASE_DIR)

print(f"Lendo CSV de: {CSV_PATH}")
df = read_csv_try_encodings(CSV_PATH)
print("Linhas x Colunas:", df.shape)
print("Colunas:", list(df.columns))

# Fonte de dados: https://drive.google.com/drive/folders/1Ubbb8K2GuwoHl9TB8VEQgAEzeKMT5nAI


In [ ]:
# 3. Visualiza81o das primeiras linhas
try:
    display(df.head(10))
except NameError:
    print("DataFrame df n34o definido. Execute a c3lula de carregamento.")


In [ ]:
# 4. Info geral: info, shape, tipos de dados, valores nulos
if 'df' in globals():
    print("Shape:", df.shape)
    print("Tipos de dados:\n", df.dtypes)
    print("\nValores nulos por coluna:\n", df.isna().sum())
    print("\nInfo:")
    buf = []
    df.info(buf)
    print("\n".join(map(str, buf)))
else:
    print("DataFrame df n34o definido.")


In [ ]:
# 5. Estat8sticas descritivas b9sicas
if 'df' in globals():
    desc_num = df.describe(include=[np.number]).T
    desc_cat = df.describe(include=['object', 'category']).T
    display(desc_num)
    display(desc_cat)
else:
    print("DataFrame df n34o definido.")


# 6. Contagem de valores ausentes por coluna (ordenado)
if 'df' in globals():
    missing = df.isna().sum().sort_values(ascending=False)
    display(missing.to_frame(name='missing'))
else:
    print("DataFrame df não definido.")


In [ ]:
# 8. Identificação de outliers com Z-Score (para colunas específicas)
if 'df_clean' in globals():
    target_cols = [
        'mandante_placar',
        'visitante_placar',
        'rodada'
    ]
    use_cols = cols_exist(df_clean, target_cols)

    outlier_flags = {}
    for c in use_cols:
        if pd.api.types.is_numeric_dtype(df_clean[c]):
            zscores = np.abs(stats.zscore(df_clean[c].astype(float), nan_policy='omit'))
            outlier_flags[c] = zscores > 3.0
        else:
            outlier_flags[c] = pd.Series(False, index=df_clean.index)

    outlier_any = pd.DataFrame(outlier_flags).any(axis=1)
    print("Total linhas com algum outlier nessas colunas:", int(outlier_any.sum()))
else:
    print("df_clean não definido.")


In [ ]:
# 9. Remoção/tratamento dos outliers
if 'df_clean' in globals():
    if 'outlier_any' in globals():
        df_no_outliers = df_clean.loc[~outlier_any].copy()
    else:
        df_no_outliers = df_clean.copy()
    print("Linhas após remover outliers:", df_no_outliers.shape[0])
else:
    print("df_clean não definido.")


In [ ]:
# 10. Normalização (Min-Max Scaling) de uma coluna numérica
from sklearn.preprocessing import MinMaxScaler

if 'df_no_outliers' in globals():
    to_scale = cols_exist(df_no_outliers, ['rodada', 'mandante_placar', 'visitante_placar'])
    if len(to_scale) > 0:
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(df_no_outliers[to_scale].astype(float))
        df_scaled = df_no_outliers.copy()
        for idx, col in enumerate(to_scale):
            df_scaled[f'{col}_minmax'] = scaled[:, idx]
        print("Colunas normalizadas adicionadas:", [f'{c}_minmax' for c in to_scale])
    else:
        df_scaled = df_no_outliers.copy()
        print("Nenhuma coluna elegível para normalização encontrada.")
else:
    print("df_no_outliers não definido.")


In [ ]:
# 4b. Info completo (correção de exibição)
from io import StringIO
if 'df' in globals():
    print("Shape:", df.shape)
    print("Tipos de dados:\n", df.dtypes)
    print("\nValores nulos por coluna:\n", df.isna().sum())
    print("\nInfo:")
    s = StringIO()
    df.info(buf=s)
    print(s.getvalue())
else:
    print("DataFrame df não definido.")


In [ ]:
# 11 (retry). One-Hot Encoding de variáveis categóricas selecionadas
if 'df_scaled' in globals():
    cat_cols = cols_exist(df_scaled, ['mandante_estado', 'visitante_estado', 'vencedor'])
    if len(cat_cols) > 0:
        df_encoded = pd.get_dummies(df_scaled, columns=cat_cols, drop_first=False, dtype=int)
        print("Colunas codificadas:", cat_cols)
    else:
        df_encoded = df_scaled.copy()
        print("Nenhuma coluna categórica selecionada encontrada.")
else:
    print("df_scaled não definido.")


In [ ]:
# 12. Matriz de correlação (focando em placares e rodadas)
if 'df_encoded' in globals():
    corr_cols = cols_exist(df_encoded, ['mandante_placar', 'visitante_placar', 'rodada'])
    corr_cols += [c for c in df_encoded.columns if c.endswith('_minmax')]
    corr_df = df_encoded[corr_cols].select_dtypes(include=[np.number])
    if corr_df.shape[1] > 0:
        corr = corr_df.corr()
        display(corr)
    else:
        print("Sem colunas numéricas para correlação.")
else:
    print("df_encoded não definido.")


In [ ]:
# 13. Gráfico de dispersão (placar mandante vs. placar visitante)
if 'df_encoded' in globals():
    xcol, ycol = 'mandante_placar', 'visitante_placar'
    if xcol in df_encoded.columns and ycol in df_encoded.columns:
        plt.figure(figsize=(6, 4))
        sns.scatterplot(data=df_encoded, x=xcol, y=ycol, alpha=0.6)
        plt.title('Mandante x Visitante (placares)')
        plt.tight_layout()
        plt.show()
    else:
        print("Colunas de placar não encontradas.")
else:
    print("df_encoded não definido.")


In [ ]:
# 14. Filtragem de dados (ex.: jogos do Flamengo como mandante)
if 'df_encoded' in globals():
    filtro_col = next((c for c in ['mandante', 'time_mandante', 'mandante_time', 'time_casa'] if c in df_encoded.columns), None)
    if filtro_col is not None and pd.api.types.is_string_dtype(df_encoded[filtro_col]):
        df_fla_mandante = df_encoded[df_encoded[filtro_col].str.contains('Flamengo', case=False, na=False)].copy()
        print("Jogos encontrados:", df_fla_mandante.shape[0])
        display(df_fla_mandante.head())
    else:
        print("Coluna de mandante não encontrada ou não textual para aplicar o filtro.")
else:
    print("df_encoded não definido.")


In [ ]:
# 15. Exportação do DataFrame processado para CSV
if 'df_encoded' in globals():
    out_path = os.path.join(BASE_DIR, 'dataset_processado.csv')
    df_encoded.to_csv(out_path, index=False)
    print("Exportado para:", out_path)
else:
    print("df_encoded não definido.")

# 16. Conversão de data e extração de ano e mês
if 'df_encoded' in globals():
    if 'data' in df_encoded.columns:
        df_encoded['data'] = try_parse_datetime(df_encoded['data'])
        df_encoded['ano'] = df_encoded['data'].dt.year
        df_encoded['mes'] = df_encoded['data'].dt.month
        print("Colunas 'ano' e 'mes' criadas.")
    else:
        print("Coluna 'data' não encontrada.")
else:
    print("df_encoded não definido.")


In [ ]:
# 17. Agrupamentos (ex.: número de vitórias por estado ou por time)
if 'df_encoded' in globals():
    # Tenta detectar uma coluna de vencedor e um possível estado do vencedor
    vencedor_col = next((c for c in ['vencedor', 'time_vencedor', 'winner'] if c in df_encoded.columns), None)
    estado_cols = [c for c in ['mandante_estado', 'visitante_estado'] if c in df_encoded.columns]

    if vencedor_col:
        vit_por_time = df_encoded.groupby(vencedor_col).size().sort_values(ascending=False)
        print("Vitórias por time (top 10):")
        display(vit_por_time.head(10))
    else:
        print("Coluna de vencedor não encontrada para o agrupamento por time.")

    if len(estado_cols) == 2 and vencedor_col:
        # Conta vitórias por estado do vencedor com base em se venceu como mandante ou visitante
        estado_vencedor = np.where(
            df_encoded[vencedor_col].eq(df_encoded.get('mandante', df_encoded.get('time_mandante', ''))),
            df_encoded['mandante_estado'] if 'mandante_estado' in df_encoded.columns else None,
            df_encoded['visitante_estado'] if 'visitante_estado' in df_encoded.columns else None
        )
        # Cria uma Series segura
        estado_vencedor = pd.Series(estado_vencedor, index=df_encoded.index)
        vit_por_estado = estado_vencedor.value_counts(dropna=True)
        print("Vitórias por estado (top 10):")
        display(vit_por_estado.head(10))
    else:
        print("Colunas de estado insuficientes para agrupamento por estado.")
else:
    print("df_encoded não definido.")


In [ ]:
# 18. Coluna derivada: diferença de gols = mandante_placar - visitante_placar
if 'df_encoded' in globals():
    if {'mandante_placar', 'visitante_placar'}.issubset(df_encoded.columns):
        df_encoded['diferenca_gols'] = df_encoded['mandante_placar'].astype(float) - df_encoded['visitante_placar'].astype(float)
        print("Coluna 'diferenca_gols' criada.")
    else:
        print("Colunas de placar não encontradas para criar a diferença de gols.")
else:
    print("df_encoded não definido.")


In [ ]:
# 19. Gráfico de barras (ex.: número de vitórias por time)
if 'df_encoded' in globals():
    vencedor_col = next((c for c in ['vencedor', 'time_vencedor', 'winner'] if c in df_encoded.columns), None)
    if vencedor_col:
        vit_por_time = df_encoded.groupby(vencedor_col).size().sort_values(ascending=False).head(15)
        plt.figure(figsize=(8, 4))
        sns.barplot(x=vit_por_time.values, y=vit_por_time.index, orient='h')
        plt.title('Vitórias por time (top 15)')
        plt.xlabel('Vitórias')
        plt.ylabel('Time')
        plt.tight_layout()
        plt.show()
    else:
        print("Coluna de vencedor não encontrada.")
else:
    print("df_encoded não definido.")
o 

In [ ]:
# 20. Heatmap da correlação
if 'df_encoded' in globals():
    corr_df = df_encoded.select_dtypes(include=[np.number])
    if corr_df.shape[1] > 1:
        plt.figure(figsize=(10, 6))
        sns.heatmap(corr_df.corr(), annot=False, cmap='coolwarm', center=0)
        plt.title('Heatmap de Correlação')
        plt.tight_layout()
        plt.show()
    else:
        print("Poucas colunas numéricas para heatmap.")
else:
    print("df_encoded não definido.")


# 7. Tratamento de valores ausentes (exemplos)
if 'df' in globals():
    df_clean = df.copy()

    # Estratégias: preencher datas, inteiros, floats e strings com abordagens sensatas
    if 'data' in df_clean.columns:
        df_clean['data'] = try_parse_datetime(df_clean['data'])

    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
    object_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()

    # Exemplo de imputação numérica: mediana
    for c in numeric_cols:
        if df_clean[c].isna().any():
            df_clean[c] = df_clean[c].fillna(df_clean[c].median())

    # Exemplo de imputação categórica: moda
    for c in object_cols:
        if df_clean[c].isna().any():
            mode_val = df_clean[c].mode(dropna=True)
            if len(mode_val) > 0:
                df_clean[c] = df_clean[c].fillna(mode_val.iloc[0])
            else:
                df_clean[c] = df_clean[c].fillna("desconhecido")

    print("Após imputação, nulos restantes: ")
    print(df_clean.isna().sum().sum())
else:
    print("DataFrame df não definido.")
